## Task 4 - Earnings Announcement Language
You’ve taken a new position at a boutique brokerage firm that wishes to focus on a “value-based” investment strategy. The firm has several analysts devoted to analyzing the extent to which quantitative financial statement information predicts future returns. You’ve been asked to consider qualitative information, particularly the narrative information accompanying firms’ earnings announcements.

Your supervisor has provided you with a sample of 2,000 earnings announcements, which data scientists (err… a PhD student) at the firm have already cleaned and parsed. You’ve also been provided with abnormal market returns surrounding each firm’s earnings announcement.

The firm would like to understand which words appear to be most predictive of the immediate investor reaction to earnings news. You have been asked to identify words conveying both “positive tone” (words that strongly predict positive returns) and “negative tone” (words that strongly predict negative returns). You remember from your applied analytics class that two finance professors, Tim Loughran and Bill McDonald, maintain a finance sentiment dictionary. You think it’d be worthwhile to consider financial sentiment from this dictionary as well.

Your task involves the following requirements:
1.	Construct a case insensitive document-term matrix using all language from the earnings announcements. Your matrix should only include the 2,000 most common words (assuming they meet the criteria below), and you should allow for bigrams *and* trigrams. You should exclude:

    a)	stop words
    
    b)	any non-alpha tokens
    
    c)	tokens shorter than 3 characters in length
    
    d)	tokens appearing in more than 90% of earnings announcements

    Report the top 25 words in your matrix.

2. Using the Loughran and McDonald financial sentiment dictionary, compute overall sentiment using three measures:

    a) positive words / total words
   
    b) negative words / total words
   
    c) (positive words – negative words) / total words

    Report descriptive statistics for these three measures (i.e., `.describe().transpose()`).

3. Compute the correlations between each word in your matrix. Answer these questions:

    a) How do each of these three measures correlate with returns?
   
    b) Are you surprised by this pattern? Why or why not?

4. Scale each row of the document term matrix by the total words in the document (i.e., so that the “counts” are proportions and sum to 1). Using these percentages, list the 25 words that correlate most positively with returns and most negatively with returns (50 total words). Report how many of these appear in the financial sentiment dictionary used in question 2.

5. (6046 only) Fit an LDA model using `sklearn` with 50 topics. Identify the topic that correlates most positively and most negative with returns. For these two topics, summarize the most relevant 10 words for each, and whether you find this pattern intuitive.

### Requirement 1
In this step, we will first load the data, and then generate the document term matrix needed to answer the specific requirements.

First, load the data and briefly inspect. There are two data files:
- `Task4_ea_sample.zip`: sample of earnings announcements
- `EA_list.csv`: Returns data for each earnings announcement

We haven't done much with zip files, so I will provide the code to load this data.


In [ ]:
import pandas as pd
from zipfile import ZipFile

folder = '/Users/cooperdenning/Documents/GitHub/MGT-6000/.venv/Task4' # Update with your own path
archive = f'{folder}/Task4_ea_sample.zip'

eadata = []
with ZipFile(archive,'r') as arc:
    for mem in arc.namelist(): #namelist() lists files in zip archive, so this loop iterates over those
        if mem.endswith('txt'):  # if the member is a txt file, it's extracted and added to the "eadata" list
            contents = arc.read(mem)
            eadata.append([mem,contents])

textdf = pd.DataFrame(eadata,columns=['File_Name','text'])
textdf['text'] = textdf['text'].str.decode('utf8')
textdf['File_Name'] = textdf['File_Name'].str.split("/").str[-1]

Now load the 'ea_list.csv' data into a dataframe called `ealist` and merge with `textdf` using the field `File_Name`:

In [3]:
ealist = pd.read_csv(f'{folder}/EA_list.csv')
both = pd.merge(ealist, textdf, how='inner',on='File_Name')

Report `info()` and `head(10)` for both:

In [4]:
# print info() and head(10) for both datasets

both.info()
both.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   File_Name      2000 non-null   object 
 1   CIK            2000 non-null   int64  
 2   datadate       2000 non-null   object 
 3   announce_date  2000 non-null   object 
 4   tic            2000 non-null   object 
 5   AbnReturn      2000 non-null   float64
 6   text           2000 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 109.5+ KB


,File_Name,CIK,datadate,announce_date,tic,AbnReturn,text
0,790705-0000950123-10-100486.txt,790705,9/30/2010,2010-11-04,TKLC,-1.826796,EX-99. 1 2 v57726exv99w1. htm EX-99. 1 exv99w...
1,814547-0001144204-14-003352.txt,814547,12/31/2013,2014-01-22,FICO,-3.337049,EX-99. 1 2 v365966_ex99-1. htm EXHIBIT 99. 1 ...
2,71829-0001437749-14-019134.txt,71829,9/30/2014,2014-10-30,NR,4.051841,EX-99 2 ex99-1. htm EXHIBIT 99. 1 ex99-1. htm...
3,71691-0001157523-19-000236.txt,71691,12/31/2018,2019-02-06,NYT,16.480962,EX-99. 1 2 a51935890ex99_1. htm EXHIBIT 99. 1 ...
4,320193-0001193125-10-230992.txt,320193,9/30/2010,2010-10-18,AAPL,-1.228231,EX-99. 1 2 dex991. htm TEXT OF PRESS RELEASE I...
5,1666134-0001171843-18-005691.txt,1666134,6/30/2018,2018-08-02,BL,10.491547,EX-99. 1 2 exh_991. htm PRESS RELEASE EdgarFi...
6,29332-0000029332-11-000064.txt,29332,6/30/2011,2011-08-04,DXYN,13.407912,EX-99. 1 2 ex99_1pressrel-01. htm 2Q 2011 PRE...
7,1318605-0001564590-20-033069.txt,1318605,6/30/2020,2020-07-22,TSLA,-9.107826,EX-99. 1 2 tsla-ex991_63. htm EX-99. 1 tsla-...
8,731012-0000950123-11-010074.txt,731012,12/31/2010,2011-02-07,HCSG,4.447709,EX-99. 1 2 c11979exv99w1. htm EXHIBIT 99. 1 E...
9,1282637-0001193125-11-103679.txt,1282637,3/31/2011,2011-04-20,NEU,13.225461,EX-99. 1 2 dex991. htm PRESS RELEASE REGARDING...


Now, construct a document term matrix using `CountVectorizer` using the specifications provided above. Your matrix should exclude:

a)	stop words (for simplicity, you can set `stop_words = 'english')`

b)	any non-alpha tokens

c)	tokens shorter than 3 characters in length

d)	tokens appearing in more than 90% of earnings announcements

Also, limit the DTM to 2,000 words, bigrams, or trigrams, and do not consider case sensitivity.


In [5]:
from sklearn.feature_extraction.text import CountVectorizer

stops = []

# Open the file in read mode
with open('/Users/cooperdenning/PycharmProjects/pythonProject5/.venv/Task4/english', 'r') as file:
    # Iterate through each line in the file
    for line in file:
        # Strip the newline character and append the line to the list
        stops.append(line.strip())

# Now, lines_list contains each line of the file as an element

vec =  CountVectorizer(
    stop_words='english',             # Exclude stop words
    token_pattern=r'\b[a-zA-Z]{3,}\b',  # Only include alpha tokens with 3+ chars
    max_df=0.9,                       # Exclude tokens in more than 90% of docs
    ngram_range=(1, 3),               # Include unigrams, bigrams, and trigrams
    max_features=2000,                # Limit vocabulary to 2000 features
    lowercase=True                    # Case insensitive processing
)
dtm = vec.fit_transform(both['text'])




FileNotFoundError: [Errno 2] No such file or directory: '/Users/cooperdenning/PycharmProjects/pythonProject5/.venv/Task4/english'

Report the top 25 words in your matrix (BONUS POINT: Report the top 25 words AND the count of each).

In [34]:
import numpy as np
vocab = vec.vocabulary_
print(vocab)
# Insert code to generate top 25 word counts (and for bonus point, count of each)
vocab_df = pd.DataFrame(list(vocab.items()), columns=["Word", "Index"])
vocab_df['Count'] = [dtm_df.iloc[:, idx].sum() for idx in vocab_df['Index']]

# Sort by count and select the top 25 words
top_25_vocab_df = vocab_df.sort_values(by='Count', ascending=False).head(25)[['Word', 'Count']]

top_25_vocab_df

{'exhibit': 658, 'announces': 118, 'revenue': 1622, 'products': 1429, 'increased': 934, 'versus': 1949, 'ago': 94, 'date': 445, 'gaap': 777, 'gross': 831, 'margin': 1088, 'eps': 613, 'november': 1246, 'nasdaq': 1170, 'data': 440, 'announced': 117, 'orders': 1304, 'order': 1303, 'primarily': 1408, 'reduction': 1543, 'decline': 458, 'markets': 1100, 'continue': 383, 'grow': 836, 'percentage': 1346, 'representing': 1585, 'september': 1690, 'backlog': 167, 'june': 995, 'margins': 1092, 'refer': 1545, 'attached': 149, 'reconciliations': 1536, 'measures': 1109, 'directly': 516, 'comparable': 307, 'basis': 180, 'reported': 1577, 'loss': 1059, 'respectively': 1599, 'president': 1396, 'chief': 266, 'executive': 655, 'officer': 1256, 'stated': 1766, 'rate': 1508, 'expected': 667, 'pleased': 1364, 'increasing': 939, 'momentum': 1161, 'solutions': 1755, 'past': 1332, 'portfolio': 1369, 'help': 845, 'customers': 439, 'manage': 1082, 'growth': 838, 'mobile': 1159, 'media': 1111, 'applications': 126,

,Word,Count
8,gaap,46793
1055,adjusted,30494
353,non gaap,28293
203,sales,27446
38,loss,25300
383,net income,24976
2,revenue,23094
388,months ended,18163
1570,ebitda,17013
33,measures,15944


### Requirement 2

Using the Loughran and McDonald financial sentiment dictionary, compute overall sentiment using three measures:

a) positive words / total words

b) negative words / total words

c) (positive words – negative words) / total words
    
First, load the LM Dictionary (file name is `LoughranMcDonald_SentimentWordLists_2018.xlsx`. Recall that the positive and negative terms are in separate sheets. Once you've loaded the language, store the positive and negative words in a list. I recommend converting to lower case at this stage.


In [36]:
lmdict = f"{folder}/LoughranMcDonald_SentimentWordLists_2018.xlsx"
lmpos = pd.read_excel(lmdict,sheet_name='Positive',header=None,names=['word'])
lmneg = pd.read_excel(lmdict,sheet_name='Negative',header=None,names=['word'])

In [39]:
# save lower case version of each word in lmpos and lmneg in two sets
pos = set(lmpos['word'].str.lower())  # Convert to lowercase and store in a set
neg = set(lmneg['word'].str.lower()) 

Second, identify the indices in the vocabulary that correspond to positive and negative words.

In [41]:
# Identify indices for positive and negative words in the vocabulary
posidx = [v for k, v in vec.vocabulary_.items() if k in pos]
negidx = [v for k, v in vec.vocabulary_.items() if k in neg]
print(negidx)

[458, 1059, 1176, 1604, 869, 520, 1852, 460, 1042, 697, 88, 1036, 872, 459, 1074, 1035, 1914, 473, 1177, 501, 476, 273, 881, 530, 277, 1956, 533, 254, 532, 481]


Third, compute the required sentiment measures and add them to the combined dataframe, `both`. Specifically add these three measures to the dataframe:

a) `pos_pct` = positive words / total words

b) `neg_pct` = negative words / total words

c) `net_pos` = (positive words – negative words) / total words

In [42]:
poswords = np.asarray(dtm[:,posidx].sum(axis=1)).flatten()
negwords = np.asarray(dtm[:,negidx].sum(axis=1)).flatten()
totwords = np.asarray(dtm.sum(axis=1)).flatten()

# Compute the sentiment measures
both['pos_pct'] = poswords / totwords
both['neg_pct'] = negwords / totwords
both['net_pos'] = (poswords - negwords) / totwords

In [46]:
both[['pos_pct', 'neg_pct', 'net_pos']].describe().transpose()

,count,mean,std,min,25%,50%,75%,max
pos_pct,2000.0,0.012726,0.005795,0.000583,0.008749,0.011907,0.015798,0.053219
neg_pct,2000.0,0.017444,0.010692,0.000000,0.009298,0.015707,0.023848,0.063321
net_pos,2000.0,-0.004718,0.011520,-0.049675,-0.011567,-0.003674,0.002980,0.041064


### Requirement 3
Answer these questions:

a) How do each of these three measures correlate with returns?
b) Are you surprised by this pattern? Why or why not?

To answer 3A, simply generate a correlation matrix with the variables needed to assess the correlation between returns and sentiment:

In [ ]:
both[...].corr()

**Answer to 3B:**
*Insert answer here*

### Requirement 4
We'll now look at how you might start to construct your own dictionary. First, scale each row of the document term matrix by the total words in the document (i.e., so that the “counts” are proportions and sum to 1).

In [ ]:
dtm_scaled = ...

Using these percentages, generate correlations between each word count and returns. List the 25 words that correlate most positively with returns and most negatively with returns (50 total words). Report how many of these appear in the financial sentiment dictionary used in requirement 2.

In [ ]:
dtm_ret = np.hstack([both['AbnReturn'].values.reshape(-1,1),dtm_scaled])
corrs = np.corrcoef(dtm_ret,rowvar=False)

List the 25 words that correlate most positively with returns and most negatively with returns (50 total words) (BONUS POINT FOR REPORTING WORD + CORRELATION). 

In [ ]:
ret_corrs = corrs[1:,0].flatten() # Grabs the first column, skipping upper left element
print("Most Negative Words")
...

In [ ]:
print("Most Positive Words")
...

Finally, report how many of these appear in the financial sentiment dictionary used in requirement 2.

In [ ]:
# Hint: See python's "intersection()" set method
print(f"There are {...} overlapping words for negative")
print(f"There are {...} overlapping words for positive")

### Requirement 5 (6046 only)

Fit an LDA model using `sklearn` with 50 topics. Set the `random_state` to 123, and leave everything else at default settings. You can use the `dtm` you generated earlier. I recomment using `fit_transform()` so you have the topic matrix ready to analyze correlations.

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation as LDA
lda = LDA(n_components=50,random_state=123)
topics = ...

Next, generate correlations between `AbnReturn` and topic weights for each of the 2,000 earnings press releases (HINT: This will be very similar to the prior step, except you do not need to scale the topics matrix by total words).

In [ ]:
top_ret = np.hstack(...)
corrs_top = ...
ret_top_corrs = ... # grab the first row or first column of corrs_top

Now, identify the topic that correlates most positively and most negatively with returns. Report the 10 words most relevant to that topics. I've provided a function you can use to access those words.

In [ ]:
def get_topic_words(topic,top_word,vocab,topn=5):
    top_words = top_word[topic,:].argsort()[-topn:][::-1].tolist() # from left to right: access row for topic, argsort, obtain final topn words, reverse the sort order
    return vocab[top_words]

mostpos = ...
mostneg = ...


In [ ]:
print("Words for most positive topic:")
print(f"{'|'.join(get_topic_words(mostpos,lda.components_,vocab,topn=10))}")

In [ ]:
print("Words for most negative topic:")
print(f"{'|'.join(get_topic_words(mostneg,lda.components_,vocab,topn=10))}")

**Final Question:** 
Do you find the topic pattern intuitive? In your opinion, does the topic-based approach appear to better identify themes that should correlate with returns?